In [9]:
# In the first cell of your Colab notebook, install required packages:
!pip install nest_asyncio crewai crewai-tools --upgrade








!pip install crewai-tools


In [ ]:
import os

# Set your OpenAI API key here (MANDATORY for CrewAI to run)
os.environ["OPENAI_API_KEY"] = "OPENAI_KEY"  # 🔁 Replace with your actual key


In [13]:



from crewai import Agent
import json

# ✅ Define Inventory Manager as a CrewAI Agent
inventory_manager = Agent(
    role="Inventory Manager",
    goal="Monitor inventory and trigger buy requests when stock is low",
    backstory=(
        "You're responsible for ensuring vendors never run out of stock. "
        "You monitor inventory levels and trigger the full 4-agent MarketplaceCrew to restock items as needed."
    ),
    allow_delegation=False,
    verbose=True
)

# ✅ Inventory agent with formatted string-style input inside the dictionary
def run_inventory_agent_string_mode():
    with open("/content/inventory.json", "r") as f:
        inventory = json.load(f)

    results = []

    print(f"\n🧠 {inventory_manager.role} ACTIVATED\n")

    for item in inventory:
        if item["current_quantity"] >= item["threshold"]:
            continue

        # Construct a formatted string for display
        input_data_string = f"ProductID={item['product_id']}, Quantity={item['desired_quantity']}, Budget={item['budget']}, Instruction='{item['instruction']}'"

        # Pass both the raw data and the string into the crew
        input_data = {
            "ProductID": item["product_id"],
            "Quantity": item["desired_quantity"],
            "Budget": item["budget"],
            "Instruction": item["instruction"],
            "InputString": input_data_string  # 🔥 You can use this string in agent prompts if needed
        }

        print(f"\n🔄 Low Stock Alert: Vendor {item['vendor_id']} | {item['name']} (Qty: {item['current_quantity']} < {item['threshold']})")
        print(f"📤 Sending buy request: {input_data_string}")

        crew = MarketplaceCrew(input_data, verbose=True)
        result = crew.run()

        results.append({
            "vendor_id": item["vendor_id"],
            "product_name": item["name"],
            "product_id": item["product_id"],
            "input_string": input_data_string,
            "result": result
        })

    return results

# ✅ Run and Print Results
final_results = run_inventory_agent_string_mode()

for res in final_results:
    print(f"\n✅ Vendor {res['vendor_id']} | {res['product_name']}")
    print(f"🧾 Request: {res['input_string']}")
    print(f"📦 Result:\n{res['result']}")



🧠 Inventory Manager ACTIVATED


🔄 Low Stock Alert: Vendor v1 | Luxury Coffee Mug (Qty: 10 < 20)
📤 Sending buy request: ProductID=p3, Quantity=30, Budget=600, Instruction='must have an elegant handle'
# Agent: Junior Seller Agent
## Task: You are given a product request and a list of sellers:

• Product Details: [{'product_id': 'p3', 'seller_id': 's13', 'name': 'Luxury Coffee Mug', 'description': 'Handcrafted ceramic mug with gold rim.', 'price': 18.0, 'quantity': 40}, {'product_id': 'p3', 'seller_id': 's14', 'name': 'Luxury Coffee Mug', 'description': 'Insulated double-walled mug for coffee enthusiasts.', 'price': 21.5, 'quantity': 60}, {'product_id': 'p3', 'seller_id': 's15', 'name': 'Luxury Coffee Mug', 'description': 'Designer coffee mug with elegant handle.', 'price': 16.99, 'quantity': 80}, {'product_id': 'p3', 'seller_id': 's16', 'name': 'Luxury Coffee Mug', 'description': 'Glossy finish ceramic mug with matching saucer.', 'price': 22.75, 'quantity': 45}, {'product_id': 'p3', 's

In [14]:
# ✅ InventoryManagerAgent to trigger your 4-agent MarketplaceCrew only for low-stock items

class InventoryManagerAgent:
    def __init__(self, inventory_file="/content/inventory.json", verbose=True):
        self.inventory_file = inventory_file
        self.verbose = verbose

    def read_inventory(self):
        with open(self.inventory_file, "r") as f:
            return json.load(f)

    def check_and_trigger_buy_requests(self):
        inventory = self.read_inventory()
        results = []

        for item in inventory:
            if item["current_quantity"] >= item["threshold"]:
                continue  # ✅ Skip if inventory is above threshold

            # Prepare input_data for MarketplaceCrew
            input_data = {
                "ProductID": item["product_id"],
                "Quantity": item["desired_quantity"],
                "Budget": item["budget"],
                "Instruction": item["instruction"]
            }

            print(f"\n🔄 Inventory low for Vendor {item['vendor_id']} | {item['name']} (Qty: {item['current_quantity']} < Threshold: {item['threshold']})")
            print(f"🧠 Sending buy request to full 4-agent crew for ProductID: {item['product_id']}")

            crew = MarketplaceCrew(input_data, verbose=self.verbose)
            result = crew.run()

            results.append({
                "vendor_id": item["vendor_id"],
                "product_name": item["name"],
                "product_id": item["product_id"],
                "result": result
            })

        return results


In [15]:
# ✅ Trigger the inventory check and purchase workflow
manager = InventoryManagerAgent()
final_results = manager.check_and_trigger_buy_requests()

# ✅ Print final results per vendor
for res in final_results:
    print(f"\n✅ Vendor {res['vendor_id']} | {res['product_name']} Result:")
    print(res["result"])



🔄 Inventory low for Vendor v1 | Luxury Coffee Mug (Qty: 10 < Threshold: 20)
🧠 Sending buy request to full 4-agent crew for ProductID: p3
# Agent: Junior Seller Agent
## Task: You are given a product request and a list of sellers:

• Product Details: [{'product_id': 'p3', 'seller_id': 's13', 'name': 'Luxury Coffee Mug', 'description': 'Handcrafted ceramic mug with gold rim.', 'price': 18.0, 'quantity': 40}, {'product_id': 'p3', 'seller_id': 's14', 'name': 'Luxury Coffee Mug', 'description': 'Insulated double-walled mug for coffee enthusiasts.', 'price': 21.5, 'quantity': 60}, {'product_id': 'p3', 'seller_id': 's15', 'name': 'Luxury Coffee Mug', 'description': 'Designer coffee mug with elegant handle.', 'price': 16.99, 'quantity': 80}, {'product_id': 'p3', 'seller_id': 's16', 'name': 'Luxury Coffee Mug', 'description': 'Glossy finish ceramic mug with matching saucer.', 'price': 22.75, 'quantity': 45}, {'product_id': 'p3', 'seller_id': 's17', 'name': 'Luxury Coffee Mug', 'description': '